# Proyecto EAF - Notebook Monolítico

## Electric Arc Furnace - Predicción de Temperatura y Composición Química

Este notebook contiene la migración completa del proyecto modular EAF a un formato monolítico.

---

## PARTE 1: Configuración del Entorno

### 1.1 Importaciones y Configuración de Logging

Importamos todas las librerías necesarias para:
- **Manipulación de datos**: pandas, numpy
- **Sistema de archivos**: os, shutil, pathlib, json
- **Logging y warnings**: para trazabilidad y control de mensajes
- **Visualización**: matplotlib, seaborn
- **Machine Learning**: sklearn (métricas, modelos, split), xgboost
- **Persistencia**: joblib para guardar/cargar modelos
- **Datos**: kagglehub para descarga de datasets

In [ ]:
# =============================================================================
# IMPORTACIONES PRINCIPALES
# =============================================================================

# Manipulación de datos
import pandas as pd
import numpy as np

# Sistema de archivos y utilidades
import os
import shutil
from pathlib import Path
import json
from typing import Dict, List, Tuple, Optional, Any

# Logging y warnings
import logging
import warnings

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Persistencia de modelos
import joblib

# Descarga de datos
import kagglehub

# Machine Learning - Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error
)
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# XGBoost
import xgboost as xgb
from xgboost import XGBRegressor

# =============================================================================
# CONFIGURACIÓN DE LOGGING
# =============================================================================

# Configurar logging para que imprima en la salida del notebook con formato de tiempo
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    force=True  # Forzar reconfiguración si ya existe
)

# Crear logger principal del notebook
logger = logging.getLogger('EAF_Notebook')
logger.setLevel(logging.INFO)

# Suprimir warnings innecesarios para limpieza de output
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
%matplotlib inline

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

logger.info("Importaciones completadas exitosamente")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"XGBoost version: {xgb.__version__}")

### 1.2 Configuración de Directorios

Definimos la estructura de carpetas del proyecto y las creamos automáticamente si no existen:
- `data/raw`: Datos crudos descargados de Kaggle
- `data/processed`: Datos procesados listos para entrenamiento
- `models`: Modelos de temperatura entrenados
- `models/chemical_results`: Modelos químicos y sus métricas

In [ ]:
# =============================================================================
# CONFIGURACIÓN DE DIRECTORIOS
# =============================================================================

# Definir raíz del proyecto usando pathlib
# Usamos Path.cwd() para notebooks, asumiendo que se ejecuta desde la raíz del proyecto
PROJECT_ROOT = Path.cwd()

# Si el notebook está en una subcarpeta, ajustar:
# PROJECT_ROOT = Path.cwd().parent  # Descomentar si es necesario

# Definir estructura de directorios
DIRECTORIES = {
    'DATA_RAW': PROJECT_ROOT / 'data' / 'raw',
    'DATA_PROCESSED': PROJECT_ROOT / 'data' / 'processed',
    'MODELS': PROJECT_ROOT / 'models',
    'CHEMICAL_RESULTS': PROJECT_ROOT / 'models' / 'chemical_results'
}

# Crear directorios si no existen
for dir_name, dir_path in DIRECTORIES.items():
    dir_path.mkdir(parents=True, exist_ok=True)
    logger.info(f"Directorio verificado/creado: {dir_name} -> {dir_path}")

# Asignar a variables individuales para fácil acceso
DATA_RAW = DIRECTORIES['DATA_RAW']
DATA_PROCESSED = DIRECTORIES['DATA_PROCESSED']
MODELS_DIR = DIRECTORIES['MODELS']
CHEMICAL_RESULTS_DIR = DIRECTORIES['CHEMICAL_RESULTS']

print(f"\n{'='*60}")
print("ESTRUCTURA DE DIRECTORIOS DEL PROYECTO")
print(f"{'='*60}")
print(f"PROJECT_ROOT:      {PROJECT_ROOT}")
print(f"DATA_RAW:          {DATA_RAW}")
print(f"DATA_PROCESSED:    {DATA_PROCESSED}")
print(f"MODELS_DIR:        {MODELS_DIR}")
print(f"CHEMICAL_RESULTS:  {CHEMICAL_RESULTS_DIR}")
print(f"{'='*60}")

### 1.3 Constantes del Negocio (CRÍTICO)

Estas constantes definen la lógica de negocio del proyecto EAF:
- **INPUT_FEATURES**: Variables de entrada del proceso (gases, carbono, materiales añadidos)
- **CHEMICAL_TARGETS**: Targets químicos a predecir (composición final del acero)
- **CHEMICAL_COLUMNS**: Columnas que requieren conversión de coma a punto decimal
- **CHEMICAL_SPECS**: Rangos de especificación para control de calidad
- **DEFAULT_HYPERPARAMS**: Hiperparámetros por defecto para los modelos
- **MODEL_DISPLAY_NAMES**: Mapeo de nombres de modelos para UI

In [ ]:
# =============================================================================
# CONSTANTES DEL NEGOCIO - PROYECTO EAF
# =============================================================================
# Migradas exactamente desde src/config.py y dashboard/config.py
# =============================================================================

# -----------------------------------------------------------------------------
# FEATURES DE INPUT
# Variables de entrada disponibles en el dataset para predicción
# -----------------------------------------------------------------------------
INPUT_FEATURES = [
    'total_o2_lance',           # Oxígeno total inyectado por lanza
    'total_gas_lance',          # Gas total inyectado por lanza
    'total_injected_carbon',    # Carbono total inyectado
    'valc',                     # Valor inicial de Carbono
    'valsi',                    # Valor inicial de Silicio
    'valmn',                    # Valor inicial de Manganeso
    'valp',                     # Valor inicial de Fósforo
    'vals',                     # Valor inicial de Azufre
    'valcu',                    # Valor inicial de Cobre
    'valcr',                    # Valor inicial de Cromo
    'valmo',                    # Valor inicial de Molibdeno
    'valni',                    # Valor inicial de Níquel
    'added_mat_140107',         # Material añadido: Cal viva
    'added_mat_202007',         # Material añadido: Mineral de hierro
    'added_mat_202008',         # Material añadido: Dolomita
    'added_mat_202039',         # Material añadido: Coque
    'added_mat_202063',         # Material añadido: Fluorita
    'added_mat_203068',         # Material añadido: FeSi
    'added_mat_203085',         # Material añadido: FeMn
    'added_mat_205069',         # Material añadido: Grafito
    'added_mat_360258',         # Material añadido: Chatarra especial
    'added_mat_705043'          # Material añadido: Otros aditivos
]

# -----------------------------------------------------------------------------
# TARGETS QUÍMICOS
# Valores FINALES de composición química a predecir
# -----------------------------------------------------------------------------
CHEMICAL_TARGETS = [
    'target_valc',              # Carbono final
    'target_valmn',             # Manganeso final
    'target_valsi',             # Silicio final
    'target_valp',              # Fósforo final
    'target_vals',              # Azufre final
    'target_valcu',             # Cobre final
    'target_valcr',             # Cromo final
    'target_valmo',             # Molibdeno final
    'target_valni'              # Níquel final
]

# -----------------------------------------------------------------------------
# TARGETS DE TEMPERATURA
# Columnas que deben excluirse de features cuando se predice temperatura
# -----------------------------------------------------------------------------
TEMPERATURE_TARGETS = ['target_temperature']

# -----------------------------------------------------------------------------
# COLUMNAS A EXCLUIR COMO FEATURES
# IDs, targets y otras columnas que no deben usarse como variables predictoras
# -----------------------------------------------------------------------------
EXCLUDE_FROM_FEATURES = [
    'heatid',                   # Identificador único de colada
    'target_temperature',       # Target de temperatura
    'target_valc',              # Target químico: Carbono
    'target_valmn',             # Target químico: Manganeso
    'target_valsi',             # Target químico: Silicio
    'target_valp',              # Target químico: Fósforo
    'target_vals',              # Target químico: Azufre
    'target_valcu',             # Target químico: Cobre
    'target_valcr',             # Target químico: Cromo
    'target_valmo',             # Target químico: Molibdeno
    'target_valni'              # Target químico: Níquel
]

# -----------------------------------------------------------------------------
# COLUMNAS QUÍMICAS
# Columnas que usan coma como separador decimal en el CSV original
# Requieren conversión a punto decimal durante la carga
# -----------------------------------------------------------------------------
CHEMICAL_COLUMNS = [
    'valc',                     # Carbono inicial
    'valsi',                    # Silicio inicial
    'valmn',                    # Manganeso inicial
    'valp',                     # Fósforo inicial
    'vals',                     # Azufre inicial
    'valcu',                    # Cobre inicial
    'valcr',                    # Cromo inicial
    'valmo',                    # Molibdeno inicial
    'valni'                     # Níquel inicial
]

# -----------------------------------------------------------------------------
# ESPECIFICACIONES QUÍMICAS
# Rangos de especificación (min, max) para valores FINALES - Control de calidad
# -----------------------------------------------------------------------------
CHEMICAL_SPECS = {
    'target_valc':  (0.05, 0.50),    # Carbono: 0.05% - 0.50%
    'target_valmn': (0.30, 1.50),    # Manganeso: 0.30% - 1.50%
    'target_valsi': (0.10, 0.60),    # Silicio: 0.10% - 0.60%
    'target_valp':  (0.001, 0.025),  # Fósforo: 0.001% - 0.025%
    'target_vals':  (0.001, 0.025),  # Azufre: 0.001% - 0.025%
    'target_valcu': (0.001, 0.030),  # Cobre: 0.001% - 0.030%
    'target_valcr': (0.001, 0.030),  # Cromo: 0.001% - 0.030%
    'target_valmo': (0.001, 0.010),  # Molibdeno: 0.001% - 0.010%
    'target_valni': (0.001, 0.030)   # Níquel: 0.001% - 0.030%
}

# -----------------------------------------------------------------------------
# RANGOS DE TEMPERATURA
# Rangos óptimos para indicador de calidad térmica
# -----------------------------------------------------------------------------
TEMPERATURE_RANGES = {
    'optimal_min': 1580,            # Temperatura mínima óptima (°C)
    'optimal_max': 1650             # Temperatura máxima óptima (°C)
}

# -----------------------------------------------------------------------------
# MODELOS DISPONIBLES
# Lista de modelos soportados para entrenamiento
# -----------------------------------------------------------------------------
AVAILABLE_MODELS = [
    'xgboost',                      # XGBoost Regressor
    'random_forest',                # Random Forest Regressor
    'linear'                        # Linear Regression
]

# -----------------------------------------------------------------------------
# NOMBRES DE MODELOS PARA DISPLAY
# Mapeo de identificadores internos a nombres para UI/reportes
# -----------------------------------------------------------------------------
MODEL_DISPLAY_NAMES = {
    'xgboost': 'XGBoost Regressor',
    'random_forest': 'Random Forest Regressor',
    'linear': 'Linear Regression'
}

# Mapeo inverso para UI -> identificador interno
UI_MODEL_NAMES = {
    'Linear Regression': 'linear',
    'Random Forest Regressor': 'random_forest',
    'XGBoost Regressor': 'xgboost'
}

# -----------------------------------------------------------------------------
# HIPERPARÁMETROS POR DEFECTO
# Configuración inicial para entrenamiento de modelos
# -----------------------------------------------------------------------------
DEFAULT_HYPERPARAMS = {
    'n_estimators': 100,            # Número de árboles (RF/XGBoost)
    'max_depth': 6,                 # Profundidad máxima de árboles
    'learning_rate': 0.1,           # Tasa de aprendizaje (XGBoost)
    'test_size': 0.2,               # Proporción de datos para test
    'random_state': 42              # Semilla para reproducibilidad
}

# -----------------------------------------------------------------------------
# DATASETS DISPONIBLES PARA EDA
# Archivos de datos procesados por tipo de análisis
# -----------------------------------------------------------------------------
EDA_DATASETS = {
    'Temperatura': 'dataset_final_temp.csv',
    'Quimica': 'dataset_final_chemical.csv'
}

# -----------------------------------------------------------------------------
# CONFIGURACIÓN DE SERVICIOS (opcional)
# URL del servicio BentoML para despliegue
# -----------------------------------------------------------------------------
BENTOML_URL = os.getenv('BENTOML_URL', 'http://localhost:3000')

# =============================================================================
# VERIFICACIÓN DE CONSTANTES
# =============================================================================
print(f"\n{'='*60}")
print("CONSTANTES DEL NEGOCIO CARGADAS")
print(f"{'='*60}")
print(f"INPUT_FEATURES:     {len(INPUT_FEATURES)} variables")
print(f"CHEMICAL_TARGETS:   {len(CHEMICAL_TARGETS)} targets")
print(f"CHEMICAL_COLUMNS:   {len(CHEMICAL_COLUMNS)} columnas")
print(f"CHEMICAL_SPECS:     {len(CHEMICAL_SPECS)} especificaciones")
print(f"AVAILABLE_MODELS:   {len(AVAILABLE_MODELS)} modelos")
print(f"{'='*60}")

print("\n--- INPUT_FEATURES (completo) ---")
for i, feat in enumerate(INPUT_FEATURES, 1):
    print(f"  {i:2d}. {feat}")

print("\n--- CHEMICAL_TARGETS (completo) ---")
for i, target in enumerate(CHEMICAL_TARGETS, 1):
    print(f"  {i}. {target}")

print("\n--- CHEMICAL_SPECS (rangos) ---")
for target, (min_val, max_val) in CHEMICAL_SPECS.items():
    print(f"  {target}: [{min_val:.3f}, {max_val:.3f}]")

logger.info("Constantes del negocio cargadas correctamente")

---

## Fin de PARTE 1

El entorno está configurado y listo para las siguientes fases:
- **PARTE 2**: Descarga y carga de datos
- **PARTE 3**: Preprocesamiento y feature engineering
- **PARTE 4**: Entrenamiento de modelos
- **PARTE 5**: Evaluación y visualización

---

## PARTE 2: Ingesta de Datos

En esta sección implementamos la descarga automática del dataset desde Kaggle.

El dataset **"Industrial Data from the Arc Furnace"** contiene 11 archivos CSV con información del proceso de fundición:
- Datos del transformador y temperatura
- Mediciones químicas iniciales y finales
- Materiales cargados e inyectados
- Datos del horno de cuchara (Ladle Furnace)

### 2.1 Configuración de Descarga y Archivos Esperados

Definimos los archivos que componen el dataset completo y la referencia al dataset de Kaggle.

In [ ]:
# =============================================================================
# CONFIGURACIÓN DE DESCARGA DE DATOS
# =============================================================================

# Dataset de Kaggle (fuente oficial de los datos)
KAGGLE_DATASET = "yuriykatser/industrial-data-from-the-arc-furnace"

# -----------------------------------------------------------------------------
# ARCHIVOS ESPERADOS DEL DATASET
# Lista completa de los 11 archivos CSV que componen el dataset
# -----------------------------------------------------------------------------
ARCHIVOS_ESPERADOS = [
    "eaf_transformer.csv",              # Datos del transformador del horno
    "basket_charged.csv",               # Cestas de chatarra cargadas
    "eaf_temp.csv",                      # Mediciones de temperatura
    "eaf_final_chemical_measurements.csv",  # Composición química final
    "eaf_added_materials.csv",           # Materiales añadidos al horno
    "inj_mat.csv",                       # Materiales inyectados
    "eaf_gaslance_mat.csv",              # Gases inyectados por lanza
    "lf_initial_chemical_measurements.csv",  # Química inicial (horno cuchara)
    "ladle_tapping.csv",                 # Datos de colada
    "lf_added_materials.csv",            # Materiales añadidos en LF
    "ferro.csv"                          # Ferroaleaciones utilizadas
]

print(f"Dataset de Kaggle: {KAGGLE_DATASET}")
print(f"Archivos esperados: {len(ARCHIVOS_ESPERADOS)}")
print("\n--- Lista de archivos ---")
for i, archivo in enumerate(ARCHIVOS_ESPERADOS, 1):
    print(f"  {i:2d}. {archivo}")

### 2.2 Función de Descarga de Datos

Implementamos `download_data()` con la siguiente lógica:
1. Verifica si todos los archivos ya existen en `data/raw`
2. Si existen y `force=False`, no descarga (evita trabajo innecesario)
3. Si faltan archivos, descarga desde Kaggle usando `kagglehub`
4. Copia los archivos al directorio del proyecto
5. Reporta el estado de cada archivo con su tamaño

In [ ]:
# =============================================================================
# FUNCIÓN DE DESCARGA DE DATOS
# =============================================================================

def download_data(force: bool = False) -> Path:
    """
    Descarga los datos de Kaggle y los copia al directorio data/raw/.
    
    Args:
        force: Si es True, sobreescribe los archivos existentes aunque ya existan.
               Si es False (default), solo descarga si faltan archivos.
    
    Returns:
        Path al directorio data/raw/ con los datos descargados.
    
    Raises:
        ConnectionError: Si hay problemas de conexión con Kaggle.
        PermissionError: Si no hay permisos para escribir en el directorio.
        Exception: Para otros errores inesperados.
    
    Example:
        >>> raw_path = download_data()  # Descarga solo si es necesario
        >>> raw_path = download_data(force=True)  # Forzar re-descarga
    """
    try:
        # Asegurar que el directorio existe
        DATA_RAW.mkdir(parents=True, exist_ok=True)
        
        # ---------------------------------------------------------------------
        # PASO 1: Verificar archivos existentes
        # ---------------------------------------------------------------------
        archivos_existentes = [
            f for f in ARCHIVOS_ESPERADOS 
            if (DATA_RAW / f).exists()
        ]
        archivos_faltantes = [
            f for f in ARCHIVOS_ESPERADOS 
            if not (DATA_RAW / f).exists()
        ]
        
        logger.info(f"Archivos existentes: {len(archivos_existentes)}/{len(ARCHIVOS_ESPERADOS)}")
        
        # Si todos existen y no se fuerza, terminar temprano
        if len(archivos_existentes) == len(ARCHIVOS_ESPERADOS) and not force:
            logger.info(f"Todos los datos ya existen en {DATA_RAW}")
            logger.info("Usa force=True para volver a descargar.")
            print(f"\n{'='*60}")
            print("DATOS YA DISPONIBLES - No se requiere descarga")
            print(f"{'='*60}")
            print(f"Directorio: {DATA_RAW}")
            print(f"Archivos: {len(archivos_existentes)}")
            return DATA_RAW
        
        # Mostrar archivos faltantes si los hay
        if archivos_faltantes and not force:
            logger.info(f"Archivos faltantes ({len(archivos_faltantes)}):")
            for archivo in archivos_faltantes:
                logger.info(f"  - {archivo}")
        
        # ---------------------------------------------------------------------
        # PASO 2: Descargar desde Kaggle
        # ---------------------------------------------------------------------
        print(f"\n{'='*60}")
        print("INICIANDO DESCARGA DESDE KAGGLE")
        print(f"{'='*60}")
        logger.info(f"Descargando dataset: {KAGGLE_DATASET}")
        logger.info("Esto puede tardar unos minutos dependiendo de la conexión...")
        
        # Descargar usando kagglehub
        kaggle_path = kagglehub.dataset_download(KAGGLE_DATASET)
        kaggle_path = Path(kaggle_path)
        
        logger.info(f"Dataset descargado en caché: {kaggle_path}")
        
        # ---------------------------------------------------------------------
        # PASO 3: Copiar archivos al directorio del proyecto
        # ---------------------------------------------------------------------
        logger.info(f"Copiando archivos a: {DATA_RAW}")
        print("-" * 60)
        
        archivos_copiados = 0
        archivos_no_encontrados = []
        total_size_mb = 0
        
        for archivo in ARCHIVOS_ESPERADOS:
            origen = kaggle_path / archivo
            destino = DATA_RAW / archivo
            
            if origen.exists():
                # Copiar archivo preservando metadatos
                shutil.copy2(origen, destino)
                
                # Calcular tamaño
                size_mb = destino.stat().st_size / (1024 * 1024)
                total_size_mb += size_mb
                
                logger.info(f"  ✓ {archivo} ({size_mb:.2f} MB)")
                archivos_copiados += 1
            else:
                archivos_no_encontrados.append(archivo)
                logger.warning(f"  ✗ {archivo} (NO ENCONTRADO en origen)")
        
        # ---------------------------------------------------------------------
        # PASO 4: Resumen final
        # ---------------------------------------------------------------------
        print("-" * 60)
        print(f"\n{'='*60}")
        print("RESUMEN DE DESCARGA")
        print(f"{'='*60}")
        print(f"Archivos copiados:    {archivos_copiados}/{len(ARCHIVOS_ESPERADOS)}")
        print(f"Tamaño total:         {total_size_mb:.2f} MB")
        print(f"Destino:              {DATA_RAW}")
        
        if archivos_no_encontrados:
            logger.warning(f"Archivos no encontrados: {archivos_no_encontrados}")
            print(f"⚠️  Archivos faltantes: {len(archivos_no_encontrados)}")
        else:
            print("✅ Todos los archivos descargados correctamente")
        
        print(f"{'='*60}")
        
        logger.info("Descarga completada exitosamente")
        return DATA_RAW
        
    except ConnectionError as e:
        logger.error(f"Error de conexión con Kaggle: {e}")
        print("❌ Error: No se pudo conectar con Kaggle.")
        print("   Verifica tu conexión a internet y credenciales de Kaggle.")
        raise
        
    except PermissionError as e:
        logger.error(f"Error de permisos: {e}")
        print(f"❌ Error: No hay permisos para escribir en {DATA_RAW}")
        raise
        
    except Exception as e:
        logger.error(f"Error inesperado durante la descarga: {e}")
        print(f"❌ Error inesperado: {e}")
        raise


def verificar_datos() -> bool:
    """
    Verifica que todos los archivos necesarios existan en data/raw.
    
    Returns:
        True si todos los archivos existen, False en caso contrario.
    
    Example:
        >>> if verificar_datos():
        ...     print("Datos listos para procesar")
    """
    if not DATA_RAW.exists():
        logger.warning(f"El directorio {DATA_RAW} no existe.")
        print("⚠️  El directorio de datos no existe.")
        print("   Ejecuta download_data() para descargar los datos.")
        return False
    
    archivos_faltantes = []
    archivos_ok = []
    
    for archivo in ARCHIVOS_ESPERADOS:
        ruta = DATA_RAW / archivo
        if ruta.exists():
            size_mb = ruta.stat().st_size / (1024 * 1024)
            archivos_ok.append((archivo, size_mb))
        else:
            archivos_faltantes.append(archivo)
    
    print(f"\n{'='*60}")
    print("VERIFICACIÓN DE DATOS")
    print(f"{'='*60}")
    
    if archivos_ok:
        print(f"\n✅ Archivos disponibles ({len(archivos_ok)}):")
        for archivo, size in archivos_ok:
            print(f"   - {archivo} ({size:.2f} MB)")
    
    if archivos_faltantes:
        print(f"\n❌ Archivos faltantes ({len(archivos_faltantes)}):")
        for archivo in archivos_faltantes:
            print(f"   - {archivo}")
        logger.warning(f"Faltan {len(archivos_faltantes)} archivos")
        return False
    
    total_size = sum(size for _, size in archivos_ok)
    print(f"\n📊 Total: {len(archivos_ok)} archivos, {total_size:.2f} MB")
    print(f"{'='*60}")
    
    logger.info(f"Verificación OK: {len(ARCHIVOS_ESPERADOS)} archivos disponibles")
    return True


print("✅ Funciones de descarga definidas: download_data(), verificar_datos()")

### 2.3 Ejecución de Descarga

Ejecutamos la función de descarga. Si los datos ya existen, no se descargará nada.
Para forzar una nueva descarga, usa `download_data(force=True)`.

In [ ]:
# =============================================================================
# EJECUCIÓN DE DESCARGA DE DATOS
# =============================================================================

# Descargar datos (solo si es necesario)
raw_data_path = download_data(force=False)

# Verificar que todos los archivos están disponibles
datos_ok = verificar_datos()

if datos_ok:
    print("\n✅ Datos listos para la siguiente fase (Preprocesamiento)")
else:
    print("\n⚠️  Ejecuta download_data(force=True) si hay problemas")

---

## Fin de PARTE 2

La ingesta de datos está completa:
- **11 archivos CSV** descargados desde Kaggle
- Datos almacenados en `data/raw/`
- Verificación automática de integridad

Siguiente fase:
- **PARTE 3**: Preprocesamiento y Feature Engineering

---

## PARTE 3: Feature Engineering y Creación del Dataset Maestro

Esta es la parte más crítica del proyecto. Transformamos los **11 archivos CSV** de series temporales en **2 datasets tabulares** listos para Machine Learning.

**Pipeline de transformación:**
1. **Carga estandarizada**: Normalizar columnas y convertir formatos europeos
2. **Agregación temporal**: Convertir series de tiempo a valores por colada (heatid)
3. **Pivotado de materiales**: Crear columnas por tipo de material añadido
4. **Extracción de targets**: Obtener temperatura y composición química final
5. **Fusión**: Combinar todas las fuentes en un dataset maestro
6. **Limpieza**: Rellenar nulos técnicos y eliminar columnas innecesarias

### 3.1 Función de Carga Estandarizada

Esta función es fundamental: carga CSVs y convierte automáticamente formatos numéricos europeos (coma decimal) a formato estándar (punto decimal).

In [ ]:
# =============================================================================
# FUNCIÓN DE CARGA ESTANDARIZADA
# =============================================================================

def load_standardized(filepath: Path) -> pd.DataFrame:
    """
    Carga un CSV y estandariza los nombres de columnas.
    
    IMPORTANTE: Detecta y convierte automáticamente formatos numéricos europeos
    donde se usa coma como separador decimal (ej: "12,5" -> 12.5).
    
    Args:
        filepath: Ruta al archivo CSV (Path o string)
    
    Returns:
        DataFrame con:
        - Columnas en minúsculas y sin espacios
        - Valores numéricos con formato decimal estándar (punto)
    
    Example:
        >>> df = load_standardized(DATA_RAW / "eaf_temp.csv")
        >>> df.columns  # ['heatid', 'temp', 'datetime', ...]
    """
    # Cargar CSV
    df = pd.read_csv(filepath, low_memory=False)
    
    # Estandarizar nombres de columnas: minúsculas y sin espacios
    df.columns = df.columns.str.lower().str.strip()
    
    # ---------------------------------------------------------------------
    # CONVERSIÓN DE COMAS DECIMALES (formato europeo -> americano)
    # Detecta columnas tipo object que contienen patrones como "12,5" o "-3,14"
    # ---------------------------------------------------------------------
    for col in df.select_dtypes(include=['object']).columns:
        # Patrón: número opcional negativo, dígitos, coma, dígitos
        # Ejemplos válidos: "12,5", "-3,14", "0,001"
        if df[col].astype(str).str.match(r'^-?\d+,\d+$').any():
            df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
            logger.debug(f"  Convertida coma decimal en columna: {col}")
    
    return df


# Test de la función
print("✅ Función load_standardized() definida")
print("\nCaracterísticas:")
print("  - Convierte nombres de columnas a minúsculas")
print("  - Elimina espacios en nombres de columnas")
print("  - Detecta y convierte comas decimales europeas a puntos")

### 3.2 Funciones de Agregación de Series Temporales

Los datos originales son series temporales (múltiples registros por colada). Estas funciones agregan los datos para obtener **un valor por colada (heatid)**.

In [ ]:
# =============================================================================
# FUNCIONES DE AGREGACIÓN DE SERIES TEMPORALES
# =============================================================================

def aggregate_gas_data(df_gas: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega los datos de gas lance por colada.
    
    Obtiene el ÚLTIMO valor registrado temporalmente (por revtime) de cada colada.
    Esto representa el estado final de O2 y gas inyectados.
    
    Args:
        df_gas: DataFrame con datos de eaf_gaslance_mat.csv
    
    Returns:
        DataFrame indexado por heatid con columnas:
        - total_o2_lance: Último valor de O2 inyectado
        - total_gas_lance: Último valor de gas inyectado
    """
    df = df_gas.copy()
    
    # Convertir columnas a numérico
    cols_gas = ['o2_amount', 'gas_amount']
    for col in cols_gas:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convertir tiempo (formato original: "2016-01-01 18:31:46,003")
    # Nota: La coma en milisegundos debe convertirse a punto
    df['revtime'] = pd.to_datetime(
        df['revtime'].astype(str).str.replace(',', '.', regex=False),
        format='%Y-%m-%d %H:%M:%S.%f',
        errors='coerce'
    )
    
    # Ordenar por tiempo y obtener el ÚLTIMO registro por colada
    df = df.sort_values('revtime')
    grp_gas = df.groupby('heatid').last()[cols_gas].rename(columns={
        'o2_amount': 'total_o2_lance',
        'gas_amount': 'total_gas_lance'
    })
    
    return grp_gas


def aggregate_injection_data(df_inj: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega los datos de inyecciones de carbón por colada.
    
    Obtiene el ÚLTIMO valor registrado temporalmente (por revtime) de cada colada.
    
    Args:
        df_inj: DataFrame con datos de inj_mat.csv
    
    Returns:
        DataFrame indexado por heatid con columna:
        - total_injected_carbon: Último valor de carbón inyectado
    """
    df = df_inj.copy()
    
    # Convertir a numérico
    df['inj_amount_carbon'] = pd.to_numeric(df['inj_amount_carbon'], errors='coerce')
    
    # Convertir tiempo (formato: "2016-01-01 18:31:46,003")
    df['revtime'] = pd.to_datetime(
        df['revtime'].astype(str).str.replace(',', '.', regex=False),
        format='%Y-%m-%d %H:%M:%S.%f',
        errors='coerce'
    )
    
    # Ordenar por tiempo y obtener el ÚLTIMO registro por colada
    df = df.sort_values('revtime')
    grp_inj = df.groupby('heatid').last()[['inj_amount_carbon']].rename(
        columns={'inj_amount_carbon': 'total_injected_carbon'}
    )
    
    return grp_inj


def aggregate_transformer_data(df_transformer: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega los datos del transformador por colada.
    
    Calcula la ENERGÍA TOTAL consumida: MW * Duración (en minutos)
    y la duración total del proceso.
    
    Args:
        df_transformer: DataFrame con datos de eaf_transformer.csv
    
    Returns:
        DataFrame indexado por heatid con columnas:
        - total_energy: Suma de (MW * duración) para toda la colada
        - total_duration: Duración total en minutos
    """
    df = df_transformer.copy()
    
    # Función para parsear DURATION de formato "MM: SS" a minutos decimales
    def parse_duration(duration_str):
        """Convierte 'MM: SS' a minutos decimales."""
        try:
            duration_str = str(duration_str).strip()
            parts = duration_str.split(':')
            if len(parts) == 2:
                minutes = float(parts[0].strip())
                seconds = float(parts[1].strip())
                return minutes + seconds / 60.0
            return 0.0
        except (ValueError, AttributeError):
            return 0.0
    
    # Aplicar conversión de duración
    df['duration_minutes'] = df['duration'].apply(parse_duration)
    
    # Convertir MW a numérico
    df['mw'] = pd.to_numeric(df['mw'], errors='coerce').fillna(0)
    
    # Calcular energía = MW * duración (en minutos)
    df['energy'] = df['mw'] * df['duration_minutes']
    
    # Agregar por colada: SUMA de energía y duración
    grp_transformer = df.groupby('heatid').agg({
        'energy': 'sum',
        'duration_minutes': 'sum'
    }).rename(columns={
        'energy': 'total_energy',
        'duration_minutes': 'total_duration'
    })
    
    return grp_transformer


def aggregate_charged_amount(df_ladle: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega la cantidad total de material cargado por colada.
    
    Args:
        df_ladle: DataFrame con datos de ladle_tapping.csv
    
    Returns:
        DataFrame indexado por heatid con columna:
        - total_charged_amount: Suma total de carga
    """
    df = df_ladle.copy()
    
    # Convertir a numérico
    df['charge_amount'] = pd.to_numeric(df['charge_amount'], errors='coerce').fillna(0)
    
    # Agregar: SUMA por colada
    grp_charged = df.groupby('heatid').agg({
        'charge_amount': 'sum'
    }).rename(columns={'charge_amount': 'total_charged_amount'})
    
    return grp_charged


print("✅ Funciones de agregación definidas:")
print("  - aggregate_gas_data(): O2 y gas inyectado (último valor)")
print("  - aggregate_injection_data(): Carbón inyectado (último valor)")
print("  - aggregate_transformer_data(): Energía total (MW * tiempo)")
print("  - aggregate_charged_amount(): Carga total (suma)")

### 3.3 Función de Pivotado de Materiales

Esta función transforma la tabla de materiales añadidos en columnas individuales.
Selecciona los **top N materiales más frecuentes** y crea una columna por cada uno (`added_mat_XXXXXX`).

In [ ]:
# =============================================================================
# FUNCIÓN DE PIVOTADO DE MATERIALES
# =============================================================================

def pivot_materials(df_ladle: pd.DataFrame, top_n: int = 10) -> pd.DataFrame:
    """
    Pivota los materiales agregados, seleccionando los top_n más frecuentes.
    
    Transforma una tabla con múltiples filas por colada (una por material)
    en una tabla con UNA fila por colada y UNA columna por material.
    
    Args:
        df_ladle: DataFrame con datos de ladle_tapping.csv
                  Debe contener columnas: heatid, mat_code, charge_amount
        top_n: Número de materiales más frecuentes a incluir (default: 10)
    
    Returns:
        DataFrame pivotado indexado por heatid con columnas:
        - added_mat_XXXXXX: Cantidad de material con código XXXXXX
        
    Example:
        >>> pivot = pivot_materials(df_ladle, top_n=10)
        >>> pivot.columns
        Index(['added_mat_140107', 'added_mat_202007', ...])
    """
    df = df_ladle.copy()
    
    # Convertir cantidad a numérico
    df['charge_amount'] = pd.to_numeric(df['charge_amount'], errors='coerce')
    
    # Seleccionar los TOP N materiales por frecuencia de uso
    top_materials = df['mat_code'].value_counts().head(top_n).index
    logger.info(f"Top {top_n} materiales seleccionados: {list(top_materials)}")
    
    # Filtrar solo los materiales más frecuentes
    df_filtered = df[df['mat_code'].isin(top_materials)]
    
    # Crear pivot table
    # - index: heatid (una fila por colada)
    # - columns: mat_code (una columna por material)
    # - values: charge_amount (suma de cantidades)
    # - fill_value: 0 (si no se usó el material, es 0)
    pivot_ladle = df_filtered.pivot_table(
        index='heatid',
        columns='mat_code',
        values='charge_amount',
        aggfunc='sum',
        fill_value=0
    ).add_prefix('added_mat_')
    
    logger.info(f"Pivot de materiales: {pivot_ladle.shape[0]} coladas, {pivot_ladle.shape[1]} materiales")
    
    return pivot_ladle


def get_datetime_range(df_ladle: pd.DataFrame) -> pd.DataFrame:
    """
    Extrae el rango de fechas (inicio y fin) de cada colada.
    
    Útil para análisis temporal y filtrado por período.
    
    Args:
        df_ladle: DataFrame con datos de ladle_tapping.csv
    
    Returns:
        DataFrame con columnas:
        - heatid: Identificador de colada
        - fecha_inicio: Primera fecha registrada
        - fecha_fin: Última fecha registrada
    """
    df = df_ladle.copy()
    
    # Convertir a datetime
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
    
    # Agregar: mínimo y máximo por colada
    datetime_range = df.groupby('heatid').agg({
        'datetime': ['min', 'max']
    })
    datetime_range.columns = ['fecha_inicio', 'fecha_fin']
    datetime_range = datetime_range.reset_index()
    
    return datetime_range


print("✅ Funciones de pivotado definidas:")
print("  - pivot_materials(): Crea columnas por material (top N)")
print("  - get_datetime_range(): Extrae rango de fechas por colada")

### 3.4 Funciones de Extracción de Targets

Estas funciones extraen las variables objetivo (targets) que queremos predecir:
- **Temperatura final**: La última medición de temperatura antes del vaciado
- **Composición química final**: Valores de C, Mn, Si, P, S, Cu, Cr, Mo, Ni

In [ ]:
# =============================================================================
# FUNCIONES DE EXTRACCIÓN DE TARGETS
# =============================================================================

def get_final_temperature(df_temp: pd.DataFrame) -> pd.DataFrame:
    """
    Obtiene la temperatura final (al vaciado) de cada colada.
    
    Toma la ÚLTIMA medición de temperatura registrada temporalmente,
    que corresponde al momento del vaciado del horno.
    
    Args:
        df_temp: DataFrame con datos de eaf_temp.csv
    
    Returns:
        DataFrame con columnas:
        - heatid: Identificador de colada
        - target_temperature: Temperatura final en °C
    """
    df = df_temp.copy()
    
    # Detectar columnas automáticamente
    cols_temp = [c for c in df.columns if 'temp' in c and 'time' not in c]
    cols_time = [c for c in df.columns if 'time' in c or 'date' in c]
    
    col_temp_name = cols_temp[0] if cols_temp else 'temp'
    col_time_name = cols_time[0] if cols_time else 'datetime'
    
    logger.info(f"Columna de temperatura detectada: {col_temp_name}")
    logger.info(f"Columna de tiempo detectada: {col_time_name}")
    
    # Limpiar tipos
    df[col_temp_name] = pd.to_numeric(df[col_temp_name], errors='coerce')
    df[col_time_name] = pd.to_datetime(df[col_time_name], errors='coerce')
    
    # Obtener la ÚLTIMA medición (temperatura al vaciado)
    # Ordenar por tiempo y tomar el último registro de cada colada
    df_target = df.sort_values(col_time_name).groupby('heatid').tail(1)
    
    # Seleccionar solo ID y temperatura
    df_target = df_target[['heatid', col_temp_name]].rename(
        columns={col_temp_name: 'target_temperature'}
    )
    
    logger.info(f"Temperaturas extraídas: {len(df_target)} coladas")
    
    return df_target


def get_final_chemical_composition(df_chem_final: pd.DataFrame) -> pd.DataFrame:
    """
    Obtiene la composición química final de cada colada.
    
    Extrae los valores finales de los elementos químicos relevantes
    para el control de calidad del acero.
    
    Args:
        df_chem_final: DataFrame con datos de eaf_final_chemical_measurements.csv
    
    Returns:
        DataFrame con columnas:
        - heatid: Identificador de colada
        - target_valc: Carbono final (%)
        - target_valmn: Manganeso final (%)
        - target_valsi: Silicio final (%)
        - target_valp: Fósforo final (%)
        - target_vals: Azufre final (%)
        - target_valcu: Cobre final (%)
        - target_valcr: Cromo final (%)
        - target_valmo: Molibdeno final (%)
        - target_valni: Níquel final (%)
    """
    # Elementos químicos a extraer como targets (lista completa)
    chemical_elements = [
        'valc', 'valmn', 'valsi', 'valp', 'vals',
        'valcu', 'valcr', 'valmo', 'valni'
    ]
    
    # Verificar qué columnas existen realmente en el archivo
    available_elements = [col for col in chemical_elements if col in df_chem_final.columns]
    
    if not available_elements:
        logger.warning("No se encontraron columnas de elementos químicos en el archivo")
        return pd.DataFrame()
    
    logger.info(f"Elementos químicos disponibles: {available_elements}")
    
    # Seleccionar heatid y elementos químicos
    cols_to_select = ['heatid'] + available_elements
    df_targets = df_chem_final[cols_to_select].copy()
    
    # Convertir a numérico (maneja tanto puntos como comas)
    for col in available_elements:
        df_targets[col] = pd.to_numeric(df_targets[col], errors='coerce')
    
    # Renombrar con prefijo target_
    rename_dict = {col: f'target_{col}' for col in available_elements}
    df_targets = df_targets.rename(columns=rename_dict)
    
    # Eliminar duplicados por heatid (tomar el último registro)
    df_targets = df_targets.drop_duplicates(subset=['heatid'], keep='last')
    
    logger.info(f"Targets químicos extraídos: {len(df_targets)} coladas, {len(available_elements)} elementos")
    
    return df_targets


print("✅ Funciones de targets definidas:")
print("  - get_final_temperature(): Última temperatura por colada")
print("  - get_final_chemical_composition(): Composición química final (9 elementos)")

### 3.5 Construcción del Dataset Maestro

Estas funciones combinan todas las fuentes de datos en un único dataset maestro, realizando los merges necesarios y la limpieza final.

In [ ]:
# =============================================================================
# CONSTRUCCIÓN DEL DATASET MAESTRO
# =============================================================================

def build_master_dataset(raw_data_dir: Path) -> pd.DataFrame:
    """
    Construye el dataset maestro combinando todas las fuentes de datos.
    
    Este es el NÚCLEO del feature engineering. Carga todos los archivos,
    aplica las agregaciones y fusiona todo en un único DataFrame.
    
    Args:
        raw_data_dir: Ruta al directorio con los datos raw
    
    Returns:
        DataFrame maestro con todas las features de input (sin targets)
        
    Pipeline:
        1. Cargar archivos CSV estandarizados
        2. Agregar series temporales (gas, inyección, transformador, carga)
        3. Pivotar materiales
        4. Extraer rango de fechas
        5. Fusionar todo por heatid
        6. Rellenar nulos técnicos con 0
    """
    logger.info("=" * 50)
    logger.info("CONSTRUYENDO DATASET MAESTRO")
    logger.info("=" * 50)
    
    # -------------------------------------------------------------------------
    # PASO 1: Cargar archivos necesarios
    # -------------------------------------------------------------------------
    logger.info("Cargando archivos...")
    
    df_gas = load_standardized(raw_data_dir / "eaf_gaslance_mat.csv")
    logger.info(f"  - eaf_gaslance_mat.csv: {df_gas.shape}")
    
    df_inj = load_standardized(raw_data_dir / "inj_mat.csv")
    logger.info(f"  - inj_mat.csv: {df_inj.shape}")
    
    df_ladle = load_standardized(raw_data_dir / "ladle_tapping.csv")
    logger.info(f"  - ladle_tapping.csv: {df_ladle.shape}")
    
    df_chem_initial = load_standardized(raw_data_dir / "lf_initial_chemical_measurements.csv")
    logger.info(f"  - lf_initial_chemical_measurements.csv: {df_chem_initial.shape}")
    
    df_transformer = load_standardized(raw_data_dir / "eaf_transformer.csv")
    logger.info(f"  - eaf_transformer.csv: {df_transformer.shape}")
    
    # -------------------------------------------------------------------------
    # PASO 2: Agregar series temporales
    # -------------------------------------------------------------------------
    logger.info("\nAgregando series temporales...")
    
    grp_gas = aggregate_gas_data(df_gas)
    logger.info(f"  - Gases: {grp_gas.shape[0]} coladas")
    
    grp_inj = aggregate_injection_data(df_inj)
    logger.info(f"  - Inyecciones: {grp_inj.shape[0]} coladas")
    
    grp_transformer = aggregate_transformer_data(df_transformer)
    logger.info(f"  - Transformador: {grp_transformer.shape[0]} coladas")
    
    grp_charged = aggregate_charged_amount(df_ladle)
    logger.info(f"  - Carga: {grp_charged.shape[0]} coladas")
    
    # -------------------------------------------------------------------------
    # PASO 3: Pivotar materiales
    # -------------------------------------------------------------------------
    logger.info("\nPivotando materiales...")
    pivot_ladle = pivot_materials(df_ladle, top_n=10)
    
    # -------------------------------------------------------------------------
    # PASO 4: Extraer rango de fechas
    # -------------------------------------------------------------------------
    logger.info("\nExtrayendo rango de fechas...")
    datetime_range = get_datetime_range(df_ladle)
    
    # -------------------------------------------------------------------------
    # PASO 5: Fusionar dataset maestro
    # -------------------------------------------------------------------------
    logger.info("\nFusionando dataset maestro...")
    
    # Dataset base: mediciones químicas iniciales
    df_master = df_chem_initial.copy()
    logger.info(f"  Base (química inicial): {df_master.shape}")
    
    # Merges (left joins para preservar todos los registros base)
    df_master = df_master.merge(grp_gas, on='heatid', how='left')
    df_master = df_master.merge(grp_inj, on='heatid', how='left')
    df_master = df_master.merge(grp_transformer, on='heatid', how='left')
    df_master = df_master.merge(grp_charged, on='heatid', how='left')
    df_master = df_master.merge(pivot_ladle, on='heatid', how='left')
    df_master = df_master.merge(datetime_range, on='heatid', how='left')
    
    logger.info(f"  Después de merges: {df_master.shape}")
    
    # -------------------------------------------------------------------------
    # PASO 6: Rellenar nulos técnicos
    # -------------------------------------------------------------------------
    cols_to_fix = [
        'total_o2_lance', 'total_gas_lance', 'total_injected_carbon',
        'total_energy', 'total_duration', 'total_charged_amount'
    ]
    # Solo rellenar las columnas que existen
    cols_to_fix = [c for c in cols_to_fix if c in df_master.columns]
    df_master[cols_to_fix] = df_master[cols_to_fix].fillna(0)
    
    # Rellenar columnas de materiales con 0
    mat_cols = [c for c in df_master.columns if c.startswith('added_mat_')]
    df_master[mat_cols] = df_master[mat_cols].fillna(0)
    
    logger.info(f"\nDataset maestro (inputs): {df_master.shape}")
    logger.info(f"  - Filas: {len(df_master)}")
    logger.info(f"  - Columnas: {len(df_master.columns)}")
    
    return df_master


def add_target_temperature(df_master: pd.DataFrame, raw_data_dir: Path) -> pd.DataFrame:
    """
    Agrega la variable target de temperatura al dataset maestro.
    
    Args:
        df_master: DataFrame con inputs
        raw_data_dir: Ruta al directorio con los datos raw
    
    Returns:
        DataFrame final con inputs y target_temperature
    """
    logger.info("\nAgregando target de temperatura...")
    
    # Cargar y extraer temperatura final
    df_temp = load_standardized(raw_data_dir / "eaf_temp.csv")
    df_target = get_final_temperature(df_temp)
    
    # Merge (inner join - solo coladas con datos completos)
    df_final = df_master.merge(df_target, on='heatid', how='inner')
    
    # Limpieza: eliminar columnas innecesarias
    cols_drop = ['datetime', 'positionrow', 'filter_key_date', 'measure_time']
    df_final = df_final.drop(columns=[c for c in cols_drop if c in df_final.columns])
    
    # Eliminar filas donde el target es nulo
    df_final = df_final.dropna(subset=['target_temperature'])
    
    # Rellenar nulos restantes en inputs con 0
    df_final = df_final.fillna(0)
    
    logger.info(f"Dataset con temperatura: {df_final.shape}")
    
    return df_final


def add_target_chemical(df_master: pd.DataFrame, raw_data_dir: Path) -> pd.DataFrame:
    """
    Agrega las variables target de composición química al dataset maestro.
    
    Args:
        df_master: DataFrame con inputs
        raw_data_dir: Ruta al directorio con los datos raw
    
    Returns:
        DataFrame final con inputs y targets químicos
    """
    logger.info("\nAgregando targets de composición química...")
    
    # Cargar y extraer composición química final
    df_chem_final = load_standardized(raw_data_dir / "eaf_final_chemical_measurements.csv")
    df_targets = get_final_chemical_composition(df_chem_final)
    
    if df_targets.empty:
        raise ValueError("No se pudieron extraer los targets químicos")
    
    # Merge (inner join - solo coladas con datos completos)
    df_final = df_master.merge(df_targets, on='heatid', how='inner')
    
    # Limpieza: eliminar columnas innecesarias
    cols_drop = ['datetime', 'positionrow', 'filter_key_date', 'measure_time']
    df_final = df_final.drop(columns=[c for c in cols_drop if c in df_final.columns])
    
    # Eliminar filas donde TODOS los targets son nulos
    target_cols = [c for c in df_final.columns if c.startswith('target_')]
    df_final = df_final.dropna(subset=target_cols, how='all')
    
    # Rellenar nulos restantes con 0
    df_final = df_final.fillna(0)
    
    logger.info(f"Dataset con química: {df_final.shape}")
    logger.info(f"Targets: {target_cols}")
    
    return df_final


print("✅ Funciones de construcción definidas:")
print("  - build_master_dataset(): Crea dataset de inputs")
print("  - add_target_temperature(): Agrega target de temperatura")
print("  - add_target_chemical(): Agrega targets químicos")

### 3.6 Función Orquestadora: build_features()

Esta función principal orquesta todo el pipeline de feature engineering y genera los dos datasets finales listos para Machine Learning.

In [ ]:
# =============================================================================
# FUNCIÓN ORQUESTADORA: BUILD_FEATURES
# =============================================================================

def build_features(force: bool = False) -> Tuple[Optional[Path], Optional[Path]]:
    """
    Pipeline completo para construir los datasets finales.
    
    Orquesta todo el proceso de feature engineering:
    1. Verifica que existan los datos raw
    2. Construye el dataset maestro de inputs
    3. Genera dataset_final_temp.csv (para predicción de temperatura)
    4. Genera dataset_final_chemical.csv (para predicción de composición química)
    
    Args:
        force: Si es True, reconstruye aunque existan los archivos.
               Si es False (default), salta si ya existen.
    
    Returns:
        Tuple con (path_temp, path_chemical):
        - path_temp: Ruta al dataset de temperatura
        - path_chemical: Ruta al dataset químico
    
    Example:
        >>> path_temp, path_chem = build_features()
        >>> path_temp, path_chem = build_features(force=True)  # Reconstruir
    """
    # Definir rutas
    raw_data_dir = DATA_RAW
    processed_data_dir = DATA_PROCESSED
    
    output_temp = processed_data_dir / "dataset_final_temp.csv"
    output_chem = processed_data_dir / "dataset_final_chemical.csv"
    
    # -------------------------------------------------------------------------
    # Verificar datos raw
    # -------------------------------------------------------------------------
    if not raw_data_dir.exists():
        raise FileNotFoundError(
            f"No existe el directorio {raw_data_dir}. "
            "Ejecuta primero download_data() para descargar los datos."
        )
    
    # Crear directorio de salida
    processed_data_dir.mkdir(parents=True, exist_ok=True)
    
    # -------------------------------------------------------------------------
    # Verificar si ya existen los datasets
    # -------------------------------------------------------------------------
    if output_temp.exists() and output_chem.exists() and not force:
        logger.info("Los datasets ya existen. Usa force=True para reconstruir.")
        print(f"\n{'='*60}")
        print("DATASETS YA DISPONIBLES")
        print(f"{'='*60}")
        print(f"  - Temperatura: {output_temp}")
        print(f"  - Químico: {output_chem}")
        print("\nUsa build_features(force=True) para reconstruir.")
        return (output_temp, output_chem)
    
    # -------------------------------------------------------------------------
    # Construir dataset maestro (solo una vez, se reutiliza)
    # -------------------------------------------------------------------------
    df_master = build_master_dataset(raw_data_dir)
    
    # -------------------------------------------------------------------------
    # Dataset de TEMPERATURA
    # -------------------------------------------------------------------------
    print(f"\n{'='*60}")
    print("CONSTRUYENDO DATASET DE TEMPERATURA")
    print(f"{'='*60}")
    
    df_temp = add_target_temperature(df_master.copy(), raw_data_dir)
    
    # Guardar
    df_temp.to_csv(output_temp, index=False)
    logger.info(f"Dataset de temperatura guardado: {output_temp}")
    
    # Mostrar resumen
    print(f"\n📊 Dataset de Temperatura:")
    print(f"   Filas: {len(df_temp):,}")
    print(f"   Columnas: {len(df_temp.columns)}")
    print(f"   Target: target_temperature")
    print(f"   Rango temperatura: [{df_temp['target_temperature'].min():.0f}, {df_temp['target_temperature'].max():.0f}] °C")
    
    # -------------------------------------------------------------------------
    # Dataset QUÍMICO
    # -------------------------------------------------------------------------
    print(f"\n{'='*60}")
    print("CONSTRUYENDO DATASET DE COMPOSICIÓN QUÍMICA")
    print(f"{'='*60}")
    
    df_chem = add_target_chemical(df_master.copy(), raw_data_dir)
    
    # Guardar
    df_chem.to_csv(output_chem, index=False)
    logger.info(f"Dataset químico guardado: {output_chem}")
    
    # Mostrar resumen
    target_cols = [c for c in df_chem.columns if c.startswith('target_')]
    print(f"\n📊 Dataset Químico:")
    print(f"   Filas: {len(df_chem):,}")
    print(f"   Columnas: {len(df_chem.columns)}")
    print(f"   Targets ({len(target_cols)}): {target_cols}")
    
    # -------------------------------------------------------------------------
    # Resumen final
    # -------------------------------------------------------------------------
    print(f"\n{'='*60}")
    print("CONSTRUCCIÓN COMPLETADA")
    print(f"{'='*60}")
    print(f"✅ Temperatura: {output_temp}")
    print(f"   Size: {output_temp.stat().st_size / 1024:.1f} KB")
    print(f"✅ Químico: {output_chem}")
    print(f"   Size: {output_chem.stat().st_size / 1024:.1f} KB")
    print(f"{'='*60}")
    
    logger.info("Feature engineering completado exitosamente")
    
    return (output_temp, output_chem)


print("✅ Función orquestadora definida: build_features(force=False)")

### 3.7 Ejecución del Feature Engineering

Ejecutamos el pipeline completo para generar los datasets procesados.

In [ ]:
# =============================================================================
# EJECUCIÓN DEL FEATURE ENGINEERING
# =============================================================================

# Ejecutar pipeline de construcción de features
# Usa force=True para reconstruir aunque ya existan
path_temp, path_chem = build_features(force=False)

# Cargar datasets para verificación
print(f"\n{'='*60}")
print("VERIFICACIÓN DE DATASETS GENERADOS")
print(f"{'='*60}")

# Dataset de temperatura
df_temp_check = pd.read_csv(path_temp)
print(f"\n📊 Dataset de Temperatura ({path_temp.name}):")
print(f"   Shape: {df_temp_check.shape}")
print(f"   Columnas: {list(df_temp_check.columns[:10])}...")
print(f"\n   Estadísticas del target:")
print(df_temp_check['target_temperature'].describe())

# Dataset químico
df_chem_check = pd.read_csv(path_chem)
target_cols = [c for c in df_chem_check.columns if c.startswith('target_')]
print(f"\n📊 Dataset Químico ({path_chem.name}):")
print(f"   Shape: {df_chem_check.shape}")
print(f"   Targets: {target_cols}")
print(f"\n   Estadísticas de targets químicos:")
print(df_chem_check[target_cols].describe().T[['mean', 'std', 'min', 'max']])

---

## Fin de PARTE 3

El Feature Engineering está completo. Se han generado:

| Dataset | Archivo | Descripción |
|---------|---------|-------------|
| Temperatura | `dataset_final_temp.csv` | Para predicción de temperatura de vaciado |
| Químico | `dataset_final_chemical.csv` | Para predicción de composición química final |

**Transformaciones aplicadas:**
- Series temporales agregadas por colada (heatid)
- Top 10 materiales pivotados como columnas
- Targets extraídos de mediciones finales
- Nulos técnicos rellenados con 0

Siguiente fase:
- **PARTE 4**: Entrenamiento de Modelos

---

## PARTE 4: Modelado de Temperatura

En esta sección implementamos el entrenamiento de modelos para predecir la **temperatura final de vaciado** del horno de arco eléctrico.

**Modelos disponibles:**
- **XGBoost Regressor**: Gradient boosting optimizado
- **Random Forest Regressor**: Ensemble de árboles de decisión
- **Linear Regression**: Modelo base lineal

**Pipeline de entrenamiento:**
1. Carga y limpieza de datos procesados
2. Filtrado de nulos y preparación de features
3. Split Train/Test (80/20)
4. Entrenamiento del modelo seleccionado
5. Evaluación con métricas (RMSE, R², MAE)
6. Visualización de resultados

### 4.1 Funciones Auxiliares de Carga y Métricas

Funciones para cargar datos procesados y calcular métricas de evaluación.

In [ ]:
# =============================================================================
# FUNCIONES AUXILIARES PARA MODELADO
# =============================================================================

def load_and_clean_data(filename: str = "dataset_final_temp.csv") -> pd.DataFrame:
    """
    Carga y limpia el dataset procesado.
    
    Realiza un DOBLE CHECK para asegurar que las columnas numéricas
    son realmente floats, convirtiendo formatos europeos si es necesario.
    
    Args:
        filename: Nombre del archivo CSV en data/processed/
    
    Returns:
        DataFrame limpio con tipos de datos correctos
    
    Raises:
        FileNotFoundError: Si no existe el archivo
    
    Example:
        >>> df = load_and_clean_data("dataset_final_temp.csv")
        >>> df = load_and_clean_data("dataset_final_chemical.csv")
    """
    # Construir ruta
    data_path = DATA_PROCESSED / filename
    
    if not data_path.exists():
        raise FileNotFoundError(
            f"No se encuentra el dataset en: {data_path}\n"
            "Ejecuta build_features() primero para generar los datos procesados."
        )
    
    # Cargar CSV
    df = pd.read_csv(data_path)
    logger.info(f"Cargado: {filename} - Shape: {df.shape}")
    
    # -------------------------------------------------------------------------
    # DOBLE CHECK: Asegurar que columnas químicas son float
    # Convierte formato europeo (coma) si es necesario
    # -------------------------------------------------------------------------
    for col in CHEMICAL_COLUMNS:
        if col in df.columns:
            # Si es object (string), convertir comas a puntos
            if df[col].dtype == 'object':
                df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
            # Convertir a numérico
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # También verificar columnas target_val*
    target_cols = [c for c in df.columns if c.startswith('target_val')]
    for col in target_cols:
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Verificar columnas numéricas principales
    numeric_cols = [
        'total_o2_lance', 'total_gas_lance', 'total_injected_carbon',
        'total_energy', 'total_duration', 'total_charged_amount',
        'target_temperature'
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df


def calculate_metrics(
    y_true: np.ndarray,
    y_pred: np.ndarray
) -> Dict[str, float]:
    """
    Calcula métricas de evaluación para regresión.
    
    Args:
        y_true: Valores reales (ground truth)
        y_pred: Valores predichos por el modelo
    
    Returns:
        Diccionario con métricas:
        - RMSE: Root Mean Squared Error
        - R2: Coeficiente de determinación
        - MAE: Mean Absolute Error
    
    Example:
        >>> metrics = calculate_metrics(y_test, y_pred)
        >>> print(f"RMSE: {metrics['RMSE']:.2f}")
    """
    y_true_arr = np.asarray(y_true)
    y_pred_arr = np.asarray(y_pred)
    
    # RMSE: Raíz del error cuadrático medio
    rmse = float(np.sqrt(mean_squared_error(y_true_arr, y_pred_arr)))
    
    # R²: Coeficiente de determinación (1.0 = perfecto)
    r2 = float(r2_score(y_true_arr, y_pred_arr))
    
    # MAE: Error absoluto medio
    mae = float(np.mean(np.abs(y_true_arr - y_pred_arr)))
    
    return {
        'RMSE': rmse,
        'R2': r2,
        'MAE': mae
    }


def get_feature_importance(
    model: Any,
    feature_names: List[str],
    model_type: str
) -> Optional[pd.DataFrame]:
    """
    Obtiene la importancia de características del modelo.
    
    Funciona con diferentes tipos de modelos:
    - Linear: Usa valores absolutos de coeficientes
    - Tree-based (RF, XGBoost): Usa feature_importances_
    
    Args:
        model: Modelo entrenado (sklearn o xgboost)
        feature_names: Lista de nombres de features
        model_type: Tipo de modelo ('xgboost', 'random_forest', 'linear')
    
    Returns:
        DataFrame con columnas 'Feature' e 'Importance',
        ordenado ascendente por importancia (para gráficos horizontales).
        Retorna None si el modelo no es soportado.
    """
    if model_type == 'linear':
        # Para regresión lineal, usar valor absoluto de coeficientes
        importance = np.abs(model.coef_)
    elif model_type in ['random_forest', 'xgboost']:
        # Para modelos de árbol, usar feature_importances_
        importance = model.feature_importances_
    else:
        logger.warning(f"Tipo de modelo no soportado para importancia: {model_type}")
        return None
    
    # Crear DataFrame ordenado
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance
    }).sort_values('Importance', ascending=True)
    
    return importance_df


print("✅ Funciones auxiliares definidas:")
print("  - load_and_clean_data(): Carga datos procesados con limpieza")
print("  - calculate_metrics(): Calcula RMSE, R², MAE")
print("  - get_feature_importance(): Extrae importancia de features")

### 4.2 Función Principal: train_temperature_model()

Esta función entrena un modelo de predicción de temperatura con el tipo y hiperparámetros especificados.

In [ ]:
# =============================================================================
# FUNCIÓN PRINCIPAL: ENTRENAMIENTO DE MODELO DE TEMPERATURA
# =============================================================================

def train_temperature_model(
    model_type: str = 'xgboost',
    n_estimators: int = None,
    max_depth: int = None,
    learning_rate: float = None,
    test_size: float = None,
    random_state: int = None,
    save_model: bool = True,
    feature_list: List[str] = None
) -> Tuple[Any, Dict[str, float], List[str], pd.DataFrame, pd.Series, np.ndarray, Optional[Path]]:
    """
    Entrena un modelo de predicción de temperatura.
    
    Args:
        model_type: Tipo de modelo ('xgboost', 'random_forest', 'linear')
        n_estimators: Número de estimadores para tree models (default: 100)
        max_depth: Profundidad máxima de árboles (default: 6)
        learning_rate: Learning rate para XGBoost (default: 0.1)
        test_size: Proporción de datos para test (default: 0.2)
        random_state: Semilla para reproducibilidad (default: 42)
        save_model: Si True, guarda el modelo en disco
        feature_list: Lista de features a usar. Si None, usa INPUT_FEATURES
    
    Returns:
        Tuple con:
        - model: Modelo entrenado
        - metrics: Dict con RMSE, R², MAE
        - feature_names: Lista de features usadas
        - X_test: DataFrame de features de test
        - y_test: Series con valores reales de test
        - y_pred: Array con predicciones
        - model_path: Path al modelo guardado (None si save_model=False)
    
    Example:
        >>> model, metrics, features, X_test, y_test, y_pred, path = train_temperature_model()
        >>> model, metrics, *_ = train_temperature_model(model_type='random_forest')
    """
    # -------------------------------------------------------------------------
    # Usar hiperparámetros por defecto si no se especifican
    # -------------------------------------------------------------------------
    n_estimators = n_estimators or DEFAULT_HYPERPARAMS['n_estimators']
    max_depth = max_depth or DEFAULT_HYPERPARAMS['max_depth']
    learning_rate = learning_rate or DEFAULT_HYPERPARAMS['learning_rate']
    test_size = test_size or DEFAULT_HYPERPARAMS['test_size']
    random_state = random_state or DEFAULT_HYPERPARAMS['random_state']
    
    print(f"\n{'='*60}")
    print(f"ENTRENAMIENTO DE MODELO DE TEMPERATURA")
    print(f"{'='*60}")
    print(f"Modelo: {MODEL_DISPLAY_NAMES.get(model_type, model_type)}")
    print(f"Hiperparámetros:")
    print(f"  - n_estimators: {n_estimators}")
    print(f"  - max_depth: {max_depth}")
    print(f"  - learning_rate: {learning_rate}")
    print(f"  - test_size: {test_size}")
    print(f"  - random_state: {random_state}")
    
    # -------------------------------------------------------------------------
    # PASO 1: Cargar datos
    # -------------------------------------------------------------------------
    logger.info("Cargando datos...")
    df = load_and_clean_data("dataset_final_temp.csv")
    
    # -------------------------------------------------------------------------
    # PASO 2: Preparar features y target
    # -------------------------------------------------------------------------
    # Usar feature_list personalizada o INPUT_FEATURES por defecto
    if feature_list is not None:
        feature_cols = [f for f in feature_list if f in df.columns]
    else:
        feature_cols = [f for f in INPUT_FEATURES if f in df.columns]
    
    X = df[feature_cols].copy()
    y = df['target_temperature'].copy()
    
    print(f"\nDataset original: {len(df)} filas")
    print(f"Features disponibles: {len(feature_cols)}")
    
    # -------------------------------------------------------------------------
    # PASO 3: Eliminar filas con valores nulos
    # -------------------------------------------------------------------------
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[mask]
    y = y[mask]
    
    print(f"Después de filtrar nulos: {len(X)} filas")
    logger.info(f"Dataset limpio: {len(X)} muestras, {len(feature_cols)} features")
    
    # -------------------------------------------------------------------------
    # PASO 4: Split Train/Test (80/20)
    # -------------------------------------------------------------------------
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    
    print(f"\nSplit Train/Test:")
    print(f"  - Train: {len(X_train)} muestras ({100*(1-test_size):.0f}%)")
    print(f"  - Test: {len(X_test)} muestras ({100*test_size:.0f}%)")
    
    # -------------------------------------------------------------------------
    # PASO 5: Crear y entrenar modelo
    # -------------------------------------------------------------------------
    logger.info(f"Entrenando modelo: {MODEL_DISPLAY_NAMES.get(model_type, model_type)}")
    
    if model_type == 'linear':
        model = LinearRegression()
    elif model_type == 'random_forest':
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=random_state,
            n_jobs=-1
        )
    elif model_type == 'xgboost':
        model = XGBRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            random_state=random_state,
            n_jobs=-1
        )
    else:
        raise ValueError(f"Modelo no reconocido: {model_type}. Use: 'xgboost', 'random_forest', 'linear'")
    
    # Entrenar
    print(f"\nEntrenando {MODEL_DISPLAY_NAMES.get(model_type, model_type)}...")
    model.fit(X_train, y_train)
    print("✅ Entrenamiento completado")
    
    # -------------------------------------------------------------------------
    # PASO 6: Predecir y evaluar
    # -------------------------------------------------------------------------
    y_pred = model.predict(X_test)
    metrics = calculate_metrics(y_test, y_pred)
    
    print(f"\n{'='*60}")
    print("MÉTRICAS DE EVALUACIÓN")
    print(f"{'='*60}")
    print(f"  RMSE: {metrics['RMSE']:.2f} °C")
    print(f"  R²:   {metrics['R2']:.4f}")
    print(f"  MAE:  {metrics['MAE']:.2f} °C")
    print(f"{'='*60}")
    
    logger.info(f"RMSE: {metrics['RMSE']:.2f}, R2: {metrics['R2']:.4f}, MAE: {metrics['MAE']:.2f}")
    
    # -------------------------------------------------------------------------
    # PASO 7: Guardar modelo (opcional)
    # -------------------------------------------------------------------------
    model_path = None
    if save_model:
        from datetime import datetime
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_name = f"temp_{model_type}_{timestamp}"
        
        # Crear subdirectorio para este modelo
        model_subdir = MODELS_DIR / model_name
        model_subdir.mkdir(exist_ok=True)
        
        model_path = model_subdir / "model.joblib"
        metadata_path = model_subdir / "metadata.json"
        
        # Guardar modelo
        joblib.dump(model, model_path)
        logger.info(f"Modelo guardado en: {model_path}")
        
        # Guardar metadatos
        metadata = {
            "model_type": model_type,
            "model_display_name": MODEL_DISPLAY_NAMES.get(model_type, model_type),
            "features": feature_cols,
            "hyperparameters": {
                "n_estimators": n_estimators,
                "max_depth": max_depth,
                "learning_rate": learning_rate,
                "test_size": test_size,
                "random_state": random_state
            },
            "metrics": metrics,
            "timestamp": timestamp,
            "target": "target_temperature",
            "n_samples_train": len(X_train),
            "n_samples_test": len(X_test)
        }
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=4)
        
        print(f"\n💾 Modelo guardado en: {model_subdir}")
    
    return model, metrics, feature_cols, X_test, y_test, y_pred, model_path


print("✅ Función principal definida: train_temperature_model()")

### 4.3 Funciones de Visualización

Funciones para generar gráficos de evaluación del modelo.

In [ ]:
# =============================================================================
# FUNCIONES DE VISUALIZACIÓN
# =============================================================================

def plot_predictions_vs_real(
    y_test: np.ndarray,
    y_pred: np.ndarray,
    metrics: Dict[str, float],
    title: str = "Predicción vs Real - Temperatura"
) -> plt.Figure:
    """
    Genera un scatter plot de predicciones vs valores reales.
    
    Args:
        y_test: Valores reales
        y_pred: Valores predichos
        metrics: Diccionario con métricas (RMSE, R², MAE)
        title: Título del gráfico
    
    Returns:
        Figure de matplotlib
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Scatter plot
    ax.scatter(y_test, y_pred, alpha=0.5, color='steelblue', edgecolors='white', linewidth=0.5)
    
    # Línea de predicción perfecta (diagonal)
    min_val = min(np.min(y_test), np.min(y_pred))
    max_val = max(np.max(y_test), np.max(y_pred))
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Predicción Perfecta')
    
    # Etiquetas y título
    ax.set_xlabel('Valor Real (°C)', fontsize=12)
    ax.set_ylabel('Valor Predicho (°C)', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    # Añadir métricas como texto
    textstr = f"RMSE: {metrics['RMSE']:.2f} °C\nR²: {metrics['R2']:.4f}\nMAE: {metrics['MAE']:.2f} °C"
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=11,
            verticalalignment='top', bbox=props)
    
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig


def plot_feature_importance(
    model: Any,
    feature_names: List[str],
    model_type: str,
    top_n: int = 15,
    title: str = "Importancia de Variables - Temperatura"
) -> plt.Figure:
    """
    Genera un bar plot horizontal de importancia de features.
    
    Args:
        model: Modelo entrenado
        feature_names: Lista de nombres de features
        model_type: Tipo de modelo ('xgboost', 'random_forest', 'linear')
        top_n: Número de features top a mostrar
        title: Título del gráfico
    
    Returns:
        Figure de matplotlib
    """
    importance_df = get_feature_importance(model, feature_names, model_type)
    
    if importance_df is None:
        print(f"⚠️ No se puede calcular importancia para modelo tipo: {model_type}")
        return None
    
    # Tomar top N features
    top_features = importance_df.tail(top_n)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Bar plot horizontal
    colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(top_features)))
    bars = ax.barh(top_features['Feature'], top_features['Importance'], color=colors)
    
    # Etiquetas y título
    ax.set_xlabel('Importancia', fontsize=12)
    ax.set_ylabel('Feature', fontsize=12)
    ax.set_title(f'{title}\n(Top {top_n} Variables)', fontsize=14, fontweight='bold')
    
    # Añadir valores en las barras
    for bar, val in zip(bars, top_features['Importance']):
        ax.text(bar.get_width() + 0.001, bar.get_y() + bar.get_height()/2,
                f'{val:.3f}', va='center', fontsize=9)
    
    ax.grid(True, axis='x', alpha=0.3)
    
    plt.tight_layout()
    return fig


def plot_residuals(
    y_test: np.ndarray,
    y_pred: np.ndarray,
    title: str = "Análisis de Residuos - Temperatura"
) -> plt.Figure:
    """
    Genera gráficos de análisis de residuos.
    
    Args:
        y_test: Valores reales
        y_pred: Valores predichos
        title: Título del gráfico
    
    Returns:
        Figure de matplotlib
    """
    residuals = np.asarray(y_test) - np.asarray(y_pred)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Subplot 1: Residuos vs Predichos
    ax1 = axes[0]
    ax1.scatter(y_pred, residuals, alpha=0.5, color='steelblue')
    ax1.axhline(y=0, color='red', linestyle='--', linewidth=2)
    ax1.set_xlabel('Valor Predicho (°C)', fontsize=11)
    ax1.set_ylabel('Residuo (°C)', fontsize=11)
    ax1.set_title('Residuos vs Predicciones', fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Subplot 2: Histograma de residuos
    ax2 = axes[1]
    ax2.hist(residuals, bins=30, color='steelblue', edgecolor='white', alpha=0.7)
    ax2.axvline(x=0, color='red', linestyle='--', linewidth=2)
    ax2.set_xlabel('Residuo (°C)', fontsize=11)
    ax2.set_ylabel('Frecuencia', fontsize=11)
    ax2.set_title('Distribución de Residuos', fontsize=12)
    
    # Añadir estadísticas
    mean_res = np.mean(residuals)
    std_res = np.std(residuals)
    textstr = f'Media: {mean_res:.2f}\nStd: {std_res:.2f}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    ax2.text(0.95, 0.95, textstr, transform=ax2.transAxes, fontsize=10,
             verticalalignment='top', horizontalalignment='right', bbox=props)
    
    fig.suptitle(title, fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    return fig


print("✅ Funciones de visualización definidas:")
print("  - plot_predictions_vs_real(): Scatter plot predicción vs real")
print("  - plot_feature_importance(): Bar plot de importancia de features")
print("  - plot_residuals(): Análisis de residuos")

### 4.4 Ejecución del Entrenamiento

Entrenamos un modelo XGBoost para predicción de temperatura y visualizamos los resultados.

In [ ]:
# =============================================================================
# ENTRENAMIENTO DEL MODELO DE TEMPERATURA
# =============================================================================

# Entrenar modelo XGBoost (por defecto)
# Puedes cambiar model_type a 'random_forest' o 'linear'
temp_model, temp_metrics, temp_features, X_test_temp, y_test_temp, y_pred_temp, temp_model_path = train_temperature_model(
    model_type='xgboost',      # Opciones: 'xgboost', 'random_forest', 'linear'
    n_estimators=100,          # Número de árboles
    max_depth=6,               # Profundidad máxima
    learning_rate=0.1,         # Learning rate
    test_size=0.2,             # 20% para test
    random_state=42,           # Semilla para reproducibilidad
    save_model=True            # Guardar modelo en disco
)

In [ ]:
# =============================================================================
# VISUALIZACIÓN DE RESULTADOS - TEMPERATURA
# =============================================================================

# Gráfico 1: Predicción vs Real
fig1 = plot_predictions_vs_real(
    y_test_temp, 
    y_pred_temp, 
    temp_metrics,
    title="Predicción vs Real - Temperatura (XGBoost)"
)
plt.show()

# Gráfico 2: Importancia de Features (Top 15)
fig2 = plot_feature_importance(
    temp_model,
    temp_features,
    model_type='xgboost',
    top_n=15,
    title="Importancia de Variables - Temperatura"
)
plt.show()

# Gráfico 3: Análisis de Residuos
fig3 = plot_residuals(
    y_test_temp,
    y_pred_temp,
    title="Análisis de Residuos - Modelo de Temperatura"
)
plt.show()

print("\n✅ Visualizaciones completadas")

---

## Fin de PARTE 4

El modelo de temperatura ha sido entrenado y evaluado:

| Métrica | Valor |
|---------|-------|
| RMSE | Error cuadrático medio |
| R² | Coeficiente de determinación |
| MAE | Error absoluto medio |

**Archivos generados:**
- `models/temp_xgboost_YYYYMMDD_HHMMSS/model.joblib`: Modelo serializado
- `models/temp_xgboost_YYYYMMDD_HHMMSS/metadata.json`: Metadatos y métricas

Siguiente fase:
- **PARTE 5**: Modelado de Composición Química

---

## PARTE 5: Modelado de Composición Química

La predicción química es más delicada debido a:
- **Outliers extremos**: Valores atípicos que distorsionan las métricas
- **Data Leakage potencial**: Usar el valor inicial del mismo elemento como feature
- **Múltiples targets**: 9 elementos químicos diferentes

**Estrategias implementadas:**
1. **Exclusión inteligente de features**: Se excluye el valor inicial del mismo elemento
2. **Capping de outliers**: Se eliminan el 1% inferior y superior (cuantiles 0.01-0.99)
3. **Imputación de NaNs**: Features con valor 0, filas con target NaN eliminadas

### 5.1 Funciones Auxiliares para Modelos Químicos

Funciones específicas para el manejo de datos químicos.

In [ ]:
# =============================================================================
# FUNCIONES AUXILIARES PARA MODELOS QUÍMICOS
# =============================================================================

def load_chemical_data() -> pd.DataFrame:
    """
    Carga el dataset específico para modelos químicos.
    
    Returns:
        DataFrame con los datos químicos procesados
    """
    return load_and_clean_data("dataset_final_chemical.csv")


def get_chemical_features(df: pd.DataFrame, target: str) -> List[str]:
    """
    Obtiene la lista de features disponibles para entrenamiento químico,
    aplicando exclusiones inteligentes para evitar data leakage.
    
    EXCLUSIONES:
    1. El propio target (target_valc, etc.)
    2. El valor inicial del mismo elemento (valc, etc.) - EVITA DATA LEAKAGE
    3. Todos los demás targets químicos
    4. Columna de identificador (heatid)
    
    Args:
        df: DataFrame con los datos
        target: Target químico a predecir (ej: 'target_valc')
    
    Returns:
        Lista de features válidas para entrenamiento
    
    Example:
        >>> features = get_chemical_features(df, 'target_valc')
        >>> # Excluirá 'valc' y 'target_valc' de las features
    """
    # Determinar el feature inicial correspondiente al target
    # target_valc -> valc, target_valmn -> valmn, etc.
    initial_feature_to_exclude = target.replace('target_', '')
    
    # Lista de columnas a excluir
    exclude_cols = ['heatid', target]
    
    # Excluir TODOS los targets químicos (no solo el actual)
    exclude_cols += [col for col in df.columns if col.startswith('target_')]
    
    # Excluir el feature inicial del mismo elemento
    if initial_feature_to_exclude in df.columns:
        exclude_cols.append(initial_feature_to_exclude)
    
    # Filtrar: usar INPUT_FEATURES que estén en df y NO estén en exclusiones
    available = [
        col for col in INPUT_FEATURES
        if col in df.columns and col not in exclude_cols
    ]
    
    return available


def cap_outliers(y: pd.Series, lower_quantile: float = 0.01, upper_quantile: float = 0.99) -> Tuple[pd.Series, int]:
    """
    Aplica capping de outliers basado en cuantiles.
    
    Elimina valores extremos que distorsionan las métricas (especialmente R²).
    
    Args:
        y: Serie con valores del target
        lower_quantile: Cuantil inferior (default: 0.01 = 1%)
        upper_quantile: Cuantil superior (default: 0.99 = 99%)
    
    Returns:
        Tuple con:
        - Máscara booleana de filas a mantener
        - Número de outliers eliminados
    
    Example:
        >>> mask, n_removed = cap_outliers(y, 0.01, 0.99)
        >>> y_clean = y[mask]
    """
    lower_bound = y.quantile(lower_quantile)
    upper_bound = y.quantile(upper_quantile)
    
    mask = (y >= lower_bound) & (y <= upper_bound)
    n_removed = (~mask).sum()
    
    return mask, n_removed


print("✅ Funciones auxiliares químicas definidas:")
print("  - load_chemical_data(): Carga dataset_final_chemical.csv")
print("  - get_chemical_features(): Selección inteligente de features")
print("  - cap_outliers(): Elimina outliers por cuantiles")

### 5.2 Función Principal: train_chemical_model()

Entrena un modelo para predecir un elemento químico específico con manejo robusto de outliers.

In [ ]:
# =============================================================================
# FUNCIÓN PRINCIPAL: ENTRENAMIENTO DE MODELO QUÍMICO
# =============================================================================

def train_chemical_model(
    target: str,
    model_type: str = 'xgboost',
    n_estimators: int = None,
    max_depth: int = None,
    learning_rate: float = None,
    test_size: float = None,
    random_state: int = None,
    save_model: bool = True,
    feature_list: List[str] = None,
    outlier_quantiles: Tuple[float, float] = (0.01, 0.99)
) -> Tuple[Any, Dict[str, float], List[str], pd.DataFrame, pd.Series, np.ndarray, Optional[Path]]:
    """
    Entrena un modelo de predicción de composición química FINAL.
    
    IMPORTANTE: Implementa limpieza de outliers para evitar R² negativos.
    
    Args:
        target: Elemento químico a predecir ('target_valc', 'target_valmn', etc.)
        model_type: Tipo de modelo ('xgboost', 'random_forest', 'linear')
        n_estimators: Número de estimadores para tree models
        max_depth: Profundidad máxima de árboles
        learning_rate: Learning rate para XGBoost
        test_size: Proporción de datos para test
        random_state: Semilla para reproducibilidad
        save_model: Si True, guarda el modelo en disco
        feature_list: Lista personalizada de features (si None, usa selección inteligente)
        outlier_quantiles: Tuple (lower, upper) para capping de outliers
    
    Returns:
        Tuple con:
        - model: Modelo entrenado
        - metrics: Dict con RMSE, R², MAE
        - feature_names: Lista de features usadas
        - X_test: DataFrame de features de test
        - y_test: Series con valores reales de test
        - y_pred: Array con predicciones
        - model_path: Path al modelo guardado
    """
    # -------------------------------------------------------------------------
    # VALIDACIÓN DEL TARGET
    # -------------------------------------------------------------------------
    if target not in CHEMICAL_TARGETS:
        raise ValueError(
            f"Target '{target}' no válido.\n"
            f"Debe ser uno de: {CHEMICAL_TARGETS}"
        )
    
    # Usar hiperparámetros por defecto si no se especifican
    n_estimators = n_estimators or DEFAULT_HYPERPARAMS['n_estimators']
    max_depth = max_depth or DEFAULT_HYPERPARAMS['max_depth']
    learning_rate = learning_rate or DEFAULT_HYPERPARAMS['learning_rate']
    test_size = test_size or DEFAULT_HYPERPARAMS['test_size']
    random_state = random_state or DEFAULT_HYPERPARAMS['random_state']
    
    # Obtener nombre limpio del elemento
    element_name = target.replace('target_', '').upper()
    
    print(f"\n{'='*60}")
    print(f"ENTRENAMIENTO DE MODELO QUÍMICO - {element_name}")
    print(f"{'='*60}")
    print(f"Target: {target}")
    print(f"Modelo: {MODEL_DISPLAY_NAMES.get(model_type, model_type)}")
    
    # Mostrar especificación si existe
    if target in CHEMICAL_SPECS:
        min_spec, max_spec = CHEMICAL_SPECS[target]
        print(f"Especificación: [{min_spec:.3f}, {max_spec:.3f}]")
    
    # -------------------------------------------------------------------------
    # PASO 1: Cargar datos
    # -------------------------------------------------------------------------
    logger.info("Cargando datos químicos...")
    df = load_chemical_data()
    
    # Verificar que el target existe
    if target not in df.columns:
        raise KeyError(
            f"La columna target '{target}' no existe en el dataset.\n"
            f"Columnas disponibles: {[c for c in df.columns if 'target' in c]}"
        )
    
    print(f"\nDataset cargado: {df.shape}")
    
    # -------------------------------------------------------------------------
    # PASO 2: Selección inteligente de features
    # -------------------------------------------------------------------------
    if feature_list is None:
        feature_cols = get_chemical_features(df, target)
    else:
        # Aplicar exclusiones a lista personalizada
        initial_feature = target.replace('target_', '')
        feature_cols = [
            f for f in feature_list
            if f in df.columns and f != target and f != initial_feature
        ]
    
    X = df[feature_cols].copy()
    y = df[target].copy()
    
    print(f"Features seleccionadas: {len(feature_cols)}")
    print(f"  (Excluido '{target.replace('target_', '')}' para evitar data leakage)")
    
    # -------------------------------------------------------------------------
    # PASO 3: Eliminar filas donde el target es NaN
    # -------------------------------------------------------------------------
    rows_initial = len(X)
    mask_not_null = y.notnull()
    X = X[mask_not_null]
    y = y[mask_not_null]
    rows_after_null = len(X)
    
    if rows_after_null < rows_initial:
        print(f"Eliminadas {rows_initial - rows_after_null} filas con target NaN")
    
    # -------------------------------------------------------------------------
    # PASO 4: CAPPING DE OUTLIERS (CRÍTICO)
    # Elimina el 1% inferior y 1% superior para evitar R² negativos
    # -------------------------------------------------------------------------
    if len(y) > 100:
        lower_q, upper_q = outlier_quantiles
        lower_bound = y.quantile(lower_q)
        upper_bound = y.quantile(upper_q)
        
        outlier_mask = (y >= lower_bound) & (y <= upper_bound)
        n_outliers = (~outlier_mask).sum()
        
        X = X[outlier_mask]
        y = y[outlier_mask]
        
        print(f"Capping de outliers ({lower_q*100:.0f}%-{upper_q*100:.0f}%): eliminados {n_outliers} outliers")
        print(f"  Rango después de capping: [{lower_bound:.4f}, {upper_bound:.4f}]")
    
    # -------------------------------------------------------------------------
    # PASO 5: Imputación de NaNs en features
    # -------------------------------------------------------------------------
    X = X.fillna(0)
    
    # Limpiar valores infinitos
    X = X.replace([np.inf, -np.inf], 0)
    y = y.replace([np.inf, -np.inf], np.nan).dropna()
    X = X.loc[y.index]  # Sincronizar índices
    
    if len(X) == 0:
        raise ValueError(f"El dataset para '{target}' quedó vacío después de la limpieza.")
    
    print(f"\nDataset final: {len(X)} muestras, {len(feature_cols)} features")
    print(f"Estadísticas del target: min={y.min():.4f}, max={y.max():.4f}, mean={y.mean():.4f}")
    
    # -------------------------------------------------------------------------
    # PASO 6: Split Train/Test
    # -------------------------------------------------------------------------
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    
    print(f"\nSplit Train/Test:")
    print(f"  - Train: {len(X_train)} muestras")
    print(f"  - Test: {len(X_test)} muestras")
    
    # -------------------------------------------------------------------------
    # PASO 7: Crear y entrenar modelo
    # -------------------------------------------------------------------------
    logger.info(f"Entrenando modelo: {MODEL_DISPLAY_NAMES.get(model_type, model_type)}")
    
    if model_type == 'linear':
        model = LinearRegression()
    elif model_type == 'random_forest':
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=random_state,
            n_jobs=-1
        )
    elif model_type == 'xgboost':
        model = XGBRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            random_state=random_state,
            n_jobs=-1
        )
    else:
        raise ValueError(f"Modelo no reconocido: {model_type}")
    
    print(f"\nEntrenando {MODEL_DISPLAY_NAMES.get(model_type, model_type)}...")
    model.fit(X_train, y_train)
    print("✅ Entrenamiento completado")
    
    # -------------------------------------------------------------------------
    # PASO 8: Predecir y evaluar
    # -------------------------------------------------------------------------
    y_pred = model.predict(X_test)
    metrics = calculate_metrics(y_test, y_pred)
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS DE EVALUACIÓN - {element_name}")
    print(f"{'='*60}")
    print(f"  RMSE: {metrics['RMSE']:.6f}")
    print(f"  R²:   {metrics['R2']:.4f}")
    print(f"  MAE:  {metrics['MAE']:.6f}")
    
    # Verificación de calidad de R²
    if metrics['R2'] < 0:
        print(f"  ⚠️  ADVERTENCIA: R² negativo. Modelo peor que la media.")
    elif metrics['R2'] < 0.3:
        print(f"  ⚠️  R² bajo. Considera ajustar hiperparámetros.")
    
    print(f"{'='*60}")
    
    # -------------------------------------------------------------------------
    # PASO 9: Guardar modelo
    # -------------------------------------------------------------------------
    model_path = None
    if save_model:
        import pickle
        from datetime import datetime
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        element_short = target.replace('target_', '')
        model_name = f"chem_{element_short}_{model_type}_{timestamp}"
        
        # Crear subdirectorio
        model_subdir = MODELS_DIR / model_name
        model_subdir.mkdir(exist_ok=True)
        
        model_path = model_subdir / "model.joblib"
        metadata_path = model_subdir / "metadata.json"
        
        # Guardar modelo
        joblib.dump(model, model_path)
        
        # Guardar metadatos
        metadata = {
            "model_type": model_type,
            "model_display_name": MODEL_DISPLAY_NAMES.get(model_type, model_type),
            "features": feature_cols,
            "hyperparameters": {
                "n_estimators": n_estimators,
                "max_depth": max_depth,
                "learning_rate": learning_rate,
                "test_size": test_size,
                "random_state": random_state
            },
            "metrics": metrics,
            "timestamp": timestamp,
            "target": target,
            "element": element_short,
            "n_samples_train": len(X_train),
            "n_samples_test": len(X_test),
            "outlier_quantiles": list(outlier_quantiles)
        }
        
        if target in CHEMICAL_SPECS:
            metadata["specification"] = {
                "min": CHEMICAL_SPECS[target][0],
                "max": CHEMICAL_SPECS[target][1]
            }
        
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=4)
        
        # Guardar también en chemical_results para compatibilidad con dashboard
        chemical_results_dir = CHEMICAL_RESULTS_DIR
        chemical_results_dir.mkdir(parents=True, exist_ok=True)
        
        importance_df = get_feature_importance(model, feature_cols, model_type)
        if importance_df is not None:
            results_data = {
                'y_test': y_test,
                'y_pred': y_pred,
                'importance_df': importance_df,
                'metrics': metrics
            }
            results_file = chemical_results_dir / f"results_{element_short}.pkl"
            with open(results_file, 'wb') as f:
                pickle.dump(results_data, f)
        
        print(f"\n💾 Modelo guardado en: {model_subdir}")
    
    return model, metrics, feature_cols, X_test, y_test, y_pred, model_path


print("✅ Función principal definida: train_chemical_model()")

### 5.3 Visualización Específica para Química

Funciones de visualización que incluyen las especificaciones químicas de calidad.

In [ ]:
# =============================================================================
# VISUALIZACIÓN ESPECÍFICA PARA QUÍMICA
# =============================================================================

def plot_chemical_predictions(
    y_test: np.ndarray,
    y_pred: np.ndarray,
    metrics: Dict[str, float],
    target: str
) -> plt.Figure:
    """
    Genera scatter plot con especificaciones químicas marcadas.
    
    Args:
        y_test: Valores reales
        y_pred: Valores predichos
        metrics: Diccionario con métricas
        target: Nombre del target (ej: 'target_valc')
    
    Returns:
        Figure de matplotlib
    """
    element_name = target.replace('target_', '').upper()
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Scatter plot
    ax.scatter(y_test, y_pred, alpha=0.5, color='steelblue', edgecolors='white', linewidth=0.5)
    
    # Línea de predicción perfecta
    min_val = min(np.min(y_test), np.min(y_pred))
    max_val = max(np.max(y_test), np.max(y_pred))
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Predicción Perfecta')
    
    # Marcar especificaciones si existen
    if target in CHEMICAL_SPECS:
        min_spec, max_spec = CHEMICAL_SPECS[target]
        ax.axhline(y=min_spec, color='green', linestyle=':', alpha=0.7, label=f'Spec Min: {min_spec}')
        ax.axhline(y=max_spec, color='green', linestyle=':', alpha=0.7, label=f'Spec Max: {max_spec}')
        ax.axvline(x=min_spec, color='green', linestyle=':', alpha=0.7)
        ax.axvline(x=max_spec, color='green', linestyle=':', alpha=0.7)
        
        # Zona de especificación
        ax.fill_between([min_spec, max_spec], min_spec, max_spec, 
                        color='green', alpha=0.1, label='Zona Óptima')
    
    ax.set_xlabel(f'Valor Real - {element_name} (%)', fontsize=12)
    ax.set_ylabel(f'Valor Predicho - {element_name} (%)', fontsize=12)
    ax.set_title(f'Predicción vs Real - {element_name}', fontsize=14, fontweight='bold')
    
    # Métricas en recuadro
    textstr = f"RMSE: {metrics['RMSE']:.6f}\nR²: {metrics['R2']:.4f}\nMAE: {metrics['MAE']:.6f}"
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
            verticalalignment='top', bbox=props)
    
    ax.legend(loc='lower right', fontsize=9)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig


def plot_chemical_summary(results_dict: Dict[str, Dict]) -> plt.Figure:
    """
    Genera un gráfico resumen de todos los modelos químicos entrenados.
    
    Args:
        results_dict: Diccionario con resultados por target
                     {target: {'metrics': {...}, 'model': ...}, ...}
    
    Returns:
        Figure de matplotlib
    """
    targets = list(results_dict.keys())
    r2_values = [results_dict[t]['metrics']['R2'] for t in targets]
    rmse_values = [results_dict[t]['metrics']['RMSE'] for t in targets]
    
    # Limpiar nombres
    labels = [t.replace('target_', '').upper() for t in targets]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Subplot 1: R² por elemento
    ax1 = axes[0]
    colors = ['green' if r2 > 0.5 else 'orange' if r2 > 0 else 'red' for r2 in r2_values]
    bars1 = ax1.bar(labels, r2_values, color=colors, edgecolor='white')
    ax1.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    ax1.axhline(y=0.5, color='green', linestyle='--', alpha=0.5, label='Umbral bueno (0.5)')
    ax1.set_ylabel('R²', fontsize=12)
    ax1.set_title('Coeficiente de Determinación (R²) por Elemento', fontsize=12)
    ax1.set_ylim(-0.5, 1.0)
    ax1.legend()
    
    # Añadir valores en barras
    for bar, val in zip(bars1, r2_values):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{val:.3f}', ha='center', fontsize=9)
    
    # Subplot 2: RMSE por elemento
    ax2 = axes[1]
    bars2 = ax2.bar(labels, rmse_values, color='steelblue', edgecolor='white')
    ax2.set_ylabel('RMSE', fontsize=12)
    ax2.set_title('Error Cuadrático Medio (RMSE) por Elemento', fontsize=12)
    
    # Añadir valores en barras
    for bar, val in zip(bars2, rmse_values):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                f'{val:.4f}', ha='center', fontsize=9, rotation=45)
    
    plt.suptitle('Resumen de Modelos Químicos', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    return fig


print("✅ Funciones de visualización química definidas:")
print("  - plot_chemical_predictions(): Scatter con especificaciones")
print("  - plot_chemical_summary(): Resumen de todos los modelos")

### 5.4 Ejecución: Entrenamiento de Modelos Químicos

Entrenamos modelos para los principales elementos químicos.

In [ ]:
# =============================================================================
# ENTRENAMIENTO DE MODELOS QUÍMICOS
# =============================================================================

# Seleccionar elementos a entrenar (principales 5 elementos)
# Puedes cambiar esta lista para incluir más o menos elementos
TARGETS_TO_TRAIN = [
    'target_valc',    # Carbono
    'target_valmn',   # Manganeso
    'target_valsi',   # Silicio
    'target_valp',    # Fósforo
    'target_vals',    # Azufre
]

# Diccionario para almacenar resultados
chemical_results = {}

print(f"{'='*60}")
print("ENTRENAMIENTO DE MODELOS QUÍMICOS")
print(f"{'='*60}")
print(f"Elementos a entrenar: {len(TARGETS_TO_TRAIN)}")
print(f"Targets: {[t.replace('target_', '').upper() for t in TARGETS_TO_TRAIN]}")
print(f"{'='*60}\n")

# Entrenar modelo para cada elemento
for target in TARGETS_TO_TRAIN:
    try:
        model, metrics, features, X_test, y_test, y_pred, path = train_chemical_model(
            target=target,
            model_type='xgboost',
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            save_model=True,
            outlier_quantiles=(0.01, 0.99)  # Eliminar 1% extremos
        )
        
        # Guardar resultados
        chemical_results[target] = {
            'model': model,
            'metrics': metrics,
            'features': features,
            'X_test': X_test,
            'y_test': y_test,
            'y_pred': y_pred,
            'path': path
        }
        
    except Exception as e:
        print(f"\n❌ Error entrenando {target}: {e}\n")
        continue

print(f"\n{'='*60}")
print(f"RESUMEN: {len(chemical_results)}/{len(TARGETS_TO_TRAIN)} modelos entrenados exitosamente")
print(f"{'='*60}")

In [ ]:
# =============================================================================
# VISUALIZACIÓN DE RESULTADOS QUÍMICOS
# =============================================================================

# Mostrar gráfico resumen de todos los modelos
if chemical_results:
    fig_summary = plot_chemical_summary(chemical_results)
    plt.show()
    
    # Tabla resumen de métricas
    print(f"\n{'='*70}")
    print("TABLA RESUMEN DE MÉTRICAS POR ELEMENTO")
    print(f"{'='*70}")
    print(f"{'Elemento':<12} {'RMSE':>12} {'R²':>12} {'MAE':>12} {'Spec Min':>10} {'Spec Max':>10}")
    print("-" * 70)
    
    for target, result in chemical_results.items():
        element = target.replace('target_', '').upper()
        metrics = result['metrics']
        
        if target in CHEMICAL_SPECS:
            min_spec, max_spec = CHEMICAL_SPECS[target]
            print(f"{element:<12} {metrics['RMSE']:>12.6f} {metrics['R2']:>12.4f} {metrics['MAE']:>12.6f} {min_spec:>10.3f} {max_spec:>10.3f}")
        else:
            print(f"{element:<12} {metrics['RMSE']:>12.6f} {metrics['R2']:>12.4f} {metrics['MAE']:>12.6f} {'N/A':>10} {'N/A':>10}")
    
    print(f"{'='*70}")
else:
    print("⚠️ No hay resultados para mostrar")

In [ ]:
# =============================================================================
# GRÁFICOS INDIVIDUALES POR ELEMENTO
# =============================================================================

# Mostrar gráfico de predicción para cada elemento entrenado
for target, result in chemical_results.items():
    fig = plot_chemical_predictions(
        result['y_test'],
        result['y_pred'],
        result['metrics'],
        target
    )
    plt.show()
    
    # Mostrar feature importance para este modelo
    fig_imp = plot_feature_importance(
        result['model'],
        result['features'],
        model_type='xgboost',
        top_n=10,
        title=f"Importancia de Variables - {target.replace('target_', '').upper()}"
    )
    if fig_imp:
        plt.show()
    
    print("-" * 60)

---

## Fin de PARTE 5

Los modelos de composición química han sido entrenados con las siguientes estrategias:

| Estrategia | Descripción |
|------------|-------------|
| **Exclusión de features** | Se excluye el valor inicial del mismo elemento para evitar data leakage |
| **Capping de outliers** | Se eliminan el 1% inferior y 1% superior (cuantiles 0.01-0.99) |
| **Imputación** | NaNs en features → 0, filas con target NaN → eliminadas |

**Archivos generados por elemento:**
- `models/chem_{elemento}_xgboost_TIMESTAMP/model.joblib`
- `models/chem_{elemento}_xgboost_TIMESTAMP/metadata.json`
- `models/chemical_results/results_{elemento}.pkl`

---

## Fin del Notebook

El notebook EAF completo incluye:
- **PARTE 1**: Configuración del entorno y constantes
- **PARTE 2**: Descarga de datos desde Kaggle
- **PARTE 3**: Feature engineering y creación del dataset maestro
- **PARTE 4**: Modelado de temperatura
- **PARTE 5**: Modelado de composición química

Todos los modelos están guardados en el directorio `models/` y listos para ser usados en el dashboard o para inferencia.